# basic ReAct(manual action)

In [1]:
import re
import os
import httpx
from openai import OpenAI

In [2]:
# 创建客户端并指向本地的 Ollama 服务
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",  # 可以是任何非空值
)

# 使用 OpenAI 兼容的 API 调用
chat_completion = client.chat.completions.create(
    model="qwen2.5:7b",
    messages=[{"role": "user", "content": "Hello world!"}]
)

In [3]:
chat_completion.choices[0].message.content

'Hello! Nice to meet you. How can I assist you today? Whether you have questions, need information, or just want to chat, feel free to let me know.'

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model="qwen2.5:7b",
            temperature=0,
            messages=self.messages)
        return completion.choices[0].message.content

react agnet 需要一个非常特定的系统消息

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [6]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [7]:
abot = Agent(prompt)

In [8]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the weight of a toy poodle using the average_dog_weight action.
Action: average_dog_weight: ToyPoodle
PAUSE


In [9]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [10]:
next_prompt = "Observation: {}".format(result)
print(next_prompt)

Observation: a toy poodles average weight is 7 lbs


In [11]:
abot(next_prompt)

'Answer: A toy poodle weighs approximately 7 lbs.'

In [12]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

## a little more complex question

In [13]:
abot = Agent(prompt)

In [14]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: To find the combined weight of the two dogs, I need to get the average weight for each breed using the `average_dog_weight` action, then sum those weights.\n\nAction: average_dog_weight: BorderCollie\nPAUSE'

In [15]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [16]:
abot(next_prompt)

'Thought: Now that I have the average weight of a Border Collie, I can proceed to get the average weight of a Scottish Terrier.\n\nAction: average_dog_weight: ScottishTerrier\nPAUSE'

In [17]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [18]:
abot(next_prompt)

'Thought: With both average weights known, I can now calculate the combined weight of a Border Collie and a Scottish Terrier by adding their individual average weights.\n\nAction: calculate: 37 + 20\nPAUSE'

In [22]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [23]:
abot(next_prompt)

'Thought: The calculation returned 57 lbs as the combined weight of a Border Collie and a Scottish Terrier.\n\nAnswer: The combined weight of your two dogs, a Border Collie and a Scottish Terrier, is 57 lbs.'